In [3]:
#  探索规则

import pandas as pd
import numpy as np

df = pd.read_csv("E:/作业/杂物/web日志分析/sampled_200k.csv")
print(f"总共有 {len(df)} 条日志记录")
print("数据的前5行：")
print(df.head())  #读取数据看一下前五行出来说明读取成功

danger = [] #创建数组用来储存可疑记录

# 逐行检查，用最简单的规则
for i, row in df.iterrows(): #遍历每一行
    ip = row['ip']
    path = row['path'] 
    status = row['status']
    user_agent = row['user_agent']
    
    score = 0    # 给这条记录打分，分数越高越可疑初始为0
    reasons = []  # 记录可疑的原因
    if len(path) > 100:
        score += 2
        reasons.append("路径太长")  # 路径太长加两分
    dangerous_words = ['script', 'union', 'select', 'drop', 'admin', '..', 'passwd']
    for word in dangerous_words:
        if word.lower() in path.lower():
            score += 3
            reasons.append(f"包含危险词:{word}")  # 包含危险词汇加三分
    if str(status).startswith('4') or str(status).startswith('5'):
        score += 1
        reasons.append("错误状态码")  # 错误状态码加4分
    if len(user_agent) < 20:
        score += 2
        reasons.append("User-Agent太短")  # User-Agent太短加2分
    special_chars = ['<', '>', '%', '&', ';']
    for char in special_chars:
        if char in path:
            score += 1
            reasons.append(f"包含特殊字符:{char}")  # 包含特殊字符加1分

    if score >= 3:
        danger.append({
            '序号': i+1,
            'IP地址': ip,
            '请求路径': path,
            '状态码': status,
            '可疑分数': score,
            '可疑原因': '; '.join(reasons)  # 如果分数超过3分，就认为可疑
        })
#  分支可以自己调整以便调整不同权重
print(f"总记录数: {len(df)}")
print(f"可疑记录数: {len(danger)}")
print(f"可疑比例: {len(danger)/len(df)*100:.1f}%")


总共有 200000 条日志记录
数据的前5行：
                ip                        time method  \
0    84.241.19.100  26/Jan/2019:17:05:54 +0330    GET   
1     2.182.230.61  25/Jan/2019:12:29:40 +0330    GET   
2  188.208.208.127  25/Jan/2019:16:53:40 +0330    GET   
3   86.106.192.128  26/Jan/2019:19:28:37 +0330   HEAD   
4     77.42.66.145  22/Jan/2019:09:07:14 +0330    GET   

                                                path  status   size  \
0                    /image/8915/specialSale?role=e1     200     11   
1                         /static/images/favicon.ico     200    152   
2                              /apple-touch-icon.png     404    392   
3  /amp_preconnect_polyfill_404_or_other_error_ex...     404      0   
4                        /image/5/productType/120x90     200  11487   

                                             referer  \
0                             https://www.zanbil.ir/   
1                                                  -   
2                                    

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("E:/作业/杂物/web日志分析/sampled_200k.csv")
print(f"总共有 {len(df)} 条日志记录")
print("数据的前5行：")
print(df.head())  #读取数据看一下前五行出来说明读取成功

danger = [] #创建数组用来储存可疑记录

# 逐行检查，用最简单的规则
for i, row in df.iterrows(): #遍历每一行
    ip = row['ip']
    path = row['path'] 
    status = row['status']
    user_agent = row['user_agent']
    
    score = 0    # 给这条记录打分，分数越高越可疑初始为0
    reasons = []  # 记录可疑的原因
    if len(path) > 100:
        score += 2
        reasons.append("路径太长")  # 路径太长加两分
    dangerous_words = ['script', 'union', 'select', 'drop', 'admin', '..', 'passwd']
    for word in dangerous_words:
        if word.lower() in path.lower():
            score += 3
            reasons.append(f"包含危险词:{word}")  # 包含危险词汇加三分
    if str(status).startswith('4') or str(status).startswith('5'):
        score += 1
        reasons.append("错误状态码")  # 错误状态码加4分
    if len(user_agent) < 20:
        score += 2
        reasons.append("User-Agent太短")  # User-Agent太短加2分
    special_chars = ['<', '>', '%', '&', ';']
    for char in special_chars:
        if char in path:
            score += 1
            reasons.append(f"包含特殊字符:{char}")  # 包含特殊字符加1分

        df.at[i, 'score'] = score
    if score >= 4:
        df.at[i, 'label'] = 1  # 高分可疑（恶意）
    elif score <= 1:
        df.at[i, 'label'] = 0  # 低分正常


In [4]:
import pandas as pd

df = pd.read_csv("E:/作业/杂物/web日志分析/sampled_200k.csv")  # 读取数据

df['score'] = 0
df['label'] = 0  # 默认都是正常的，后续高分的我们再打成 1

for i, row in df.iterrows():  #和上边的评分规则一模一样
    score = 0
    path = str(row['path']) if pd.notna(row['path']) else ""
    status = str(row['status']) if pd.notna(row['status']) else ""
    user_agent = str(row['user_agent']) if pd.notna(row['user_agent']) else ""

    if len(path) > 100:
        score += 2
    dangerous_words = ['script', 'union', 'select', 'drop', 'admin', '..', 'passwd']
    for word in dangerous_words:
        if word.lower() in path.lower():
            score += 3
    if status.startswith('4') or status.startswith('5'):
        score += 1
    if len(user_agent) < 20:
        score += 2
    special_chars = ['<', '>', '%', '&', ';']
    for char in special_chars:
        if char in path:
            score += 1

    df.at[i, 'score'] = score
    df.at[i, 'label'] = 1 if score >= 3 else 0  # 可疑打为1，正常为0


df.to_csv("E:/作业/杂物/web日志分析/sampled_200k_labeled.csv", index=False, encoding='utf-8-sig')  # 保存为新的带伪标签的数据集

print("打标签完成")


打标签完成
